In [ ]:
from getpass import getpass
from shapely.geometry import mapping

import geopandas as gpd
import pandas as pd

from helpers import stats_to_df, authenticate_sentinel_hub, build_json_request

# Authentication
To use **Sentinel Hub API**, you need to authenticate with using **OAuth2** credentials. These are available from your [Sentinel Hub Dashboard](https://apps.sentinel-hub.com/dashboard/),

Get your `CLIENT_ID` and `CLIENT_SECRET` from the **Dashboard** -> **My Account** _> **OAuth Clients**.
Get your `DATA_SET` from [Insight Platform](https://insights.planet.com/analyze/configurations/) -> **Your Collection** -> **Collection ID**

In [ ]:
CLIENT_ID = getpass("Enter your Sentinel Hub CLIENT_ID: ")
CLIENT_SECRET = getpass("Enter your Sentinel Hub CLIENT_SECRET: ")
DATA_SET = getpass("Enter you data ID")

oauth = authenticate_sentinel_hub(CLIENT_ID, CLIENT_SECRET)

# 📂 Get Your Data Ready

Provide a **GeoJSON** file containing the geographic area or features you want to work with, the start and end date for the timeframe of your analisys, and the day desired day interval

In [ ]:
data = "test_bbox_landcover.geojson"
start_date = "2023-01-01"
end_date = "2023-02-01"
day_interval = 10
results = []

In [ ]:
geodataframe = gpd.read_file(data)
geodataframe = geodataframe.to_crs("EPSG:4326")
geodataframe

,FID,Shape_Leng,Shape_Area,land_type,geometry
0,0,800,40000,urban,"POLYGON ((-83.52432 8.96078, -83.52252 8.96078..."
1,1,800,40000,palm_plantatios,"POLYGON ((-83.53869 8.96793, -83.5369 8.96793,..."
2,2,800,40000,low_vegetation,"POLYGON ((-83.55127 8.9715, -83.54947 8.9715, ..."
3,3,800,40000,forest,"POLYGON ((-83.50456 8.98579, -83.50276 8.98579..."


# 

In [ ]:
for geoshape in geodataframe.geometry.values:
  geometry = mapping(geoshape)

  json_request = build_json_request(geometry, start_date, end_date, day_interval, DATA_SET)

  # Set the request url and headers
  url = "https://services.sentinel-hub.com/api/v1/statistics"
  headers = {
  "Accept": "application/json",
  "Content-Type": "application/json"
  }

  #Send the request
  response = oauth.request(
    "POST", url, headers=headers, json = json_request
  )

  results.append(response.json())


In [6]:
dataframe = [stats_to_df(result) for result in results]

for df, land_type in zip(dataframe, geodataframe['land_type'].values):
    df["land_type"] = land_type

dataframe = pd.concat(dataframe)

dataframe

,interval_from,interval_to,eobrowserStats_B0_min,eobrowserStats_B0_max,eobrowserStats_B0_mean,eobrowserStats_B0_stDev,eobrowserStats_B0_sampleCount,eobrowserStats_B0_noDataCount,land_type
0,2023-01-01,2023-01-11,305.160004,310.440002,308.512677,1.909740,65536,0,urban
1,2023-01-11,2023-01-21,307.299988,312.839996,311.031807,2.080841,65536,0,urban
2,2023-01-21,2023-01-31,307.730011,314.649994,312.346133,2.516827,65536,0,urban
0,2023-01-01,2023-01-11,301.899994,303.140015,302.415316,0.384106,65536,0,palm_plantatios
1,2023-01-11,2023-01-21,303.769989,305.200012,304.451732,0.451925,65536,0,palm_plantatios
2,2023-01-21,2023-01-31,304.920013,306.549988,305.594139,0.452113,65536,0,palm_plantatios
0,2023-01-01,2023-01-11,304.510010,306.630005,306.040572,0.723472,65536,0,low_vegetation
1,2023-01-11,2023-01-21,306.589996,309.040009,308.125093,0.877326,65536,0,low_vegetation
2,2023-01-21,2023-01-31,307.369995,311.290009,309.609371,1.234892,65536,0,low_vegetation
0,2023-01-01,2023-01-11,301.070007,303.920013,302.934614,0.531389,65536,0,forest
